In [ ]:
!pip install spacy==3.3.0
!pip install spacy-transformers

In [1]:
import re
import glob
import json
import spacy
import spacy.cli
import pandas as pd
from spacy.tokens import DocBin
from tqdm import tqdm

# spacy.cli.download('en_core_web_sm')

In [3]:
nlp = spacy.load('en_core_web_sm')
db = DocBin()

Text Category Training

In [17]:
df = pd.read_csv('exp_train.csv')
df['text'] = df['text'].astype(str)
df['label'] = df['label'].astype(str)

,text,label
0,ALYA ATAYA 6282376745433 alyatayaagmailcom \r\...,no
1,hope to,no
2,Work Experiences,no
3,Cariilmu Jakarta\r\nManager of Customer Servic...,yes
4,Present,no
...,...,...
568,PROFESSIONAL CERTIFICATION,no
569,Professional Certification of Human Resources ...,no
570,ADDITIONAL INFORMATION,no
571,LANGUAGE SKILLS Indonesian Native Proficiency ...,no


In [23]:
train = []
for i in df.index:
    text = df['text'][i]
    train.append((text, df['label'][i]))

train

[('ALYA ATAYA 6282376745433 alyatayaagmailcom \r\n7 Anggrek 15B Karet Setiabudi am a Manager of Customer Service in Cariilmu with a problemsolving mindset reconciliation and collaborative experience\r\nvalue add and expand my horizons ',
  'no'),
 (' hope to ', 'no'),
 ('Work Experiences ', 'no'),
 ('Cariilmu Jakarta\r\nManager of Customer Service Set the customer service staff schedule Host Admin Qontak Agent e Report Qontak Assist or replace the role of Customer Service Staff when needed including replying to Qontak Scheduling Includes create zoom links telegram groups Follow Up participants who havent passed the quiz and havent done the quiz Send the data of participants who passed the quiz to the platform Maubelajarapa\r\nCariilmu Jakarta Operation Product Development Intern Host and admin prework webinars Update webinar participants through google spreadsheets Making module and conduct research related to competitors Punya Karya Jakarta Business Development intern Group listing fo

In [10]:
def make_docs(data):
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        if label == 'no':
            doc.cats['experience'] = 0
            doc.cats['no'] = 1
        else:
            doc.cats['experience'] = 1
            doc.cats['no'] = 0
        docs.append(doc)
    return docs

In [12]:
train_docs = make_docs(train)
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk('./textcat_train.spacy')

Named Entity Recognition Training

In [ ]:
data = []
train = []

for file in glob.glob('ini lokasi data jsonl'):
  f = open(file)
  data.append(json.load(f))

for i in range(len(data)):
    for j in data[0]['annotations']:
        train.append(tuple(j))

In [ ]:
for text, annot in tqdm(train):
  doc = nlp.make_doc(text)
  ents = []
  for start, end, label in annot['entities']:
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is None:
      print("Skip")
    else:
      ents.append(span)
  doc.ents = ents
  db.add(doc)

db.to_disk('./ner_data.spacy')

In [ ]:
!python -m spacy init fill-config spacy_training/base_config.cfg config.cfg

In [ ]:
!python -m spacy train config.cfg --output ./model/model_job --paths.train ./ner_data.spacy --paths.dev ./validation_data.spacy --gpu-id 0

In [ ]:
# nlp.to_disk()